In [1]:
import requests
from geopy.geocoders import Nominatim
import re

In [9]:
StrLocation = input("Enter Location:")
if re.search('[a-zA-Z]', StrLocation):
  geolocator = Nominatim(user_agent='myapplication')
  location = geolocator.geocode(StrLocation)
  latValue = location.latitude
  lonValue = location.longitude
else:
  latValue = StrLocation.split(',')[0]
  lonValue = StrLocation.split(',')[1]


Enter Location:chennai,in


In [13]:
r = requests.get('https://power.larc.nasa.gov/api/temporal/hourly/point?start=20220308&end=20220309&latitude='+str(latValue)+'&longitude='+str(lonValue)+'&community=ag&parameters=TS%2CALLSKY_SFC_UVA&format=json&header=true&time-standard=lst')

In [11]:
r.json()

{'geometry': {'coordinates': [80.270186, 13.0836939, 48.95], 'type': 'Point'},
 'header': {'api': {'name': 'POWER Hourly API', 'version': 'v2.2.20'},
  'end': '20220309',
  'fill_value': -999.0,
  'start': '20220308',
  'title': 'NASA/POWER CERES/MERRA2 Native Resolution Hourly Data'},
 'messages': [],
 'parameters': {'ALLSKY_SFC_UVA': {'longname': 'All Sky Surface UVA Irradiance',
   'units': 'W/m^2'},
  'TS': {'longname': 'Earth Skin Temperature', 'units': 'C'}},
 'properties': {'parameter': {'ALLSKY_SFC_UVA': {'2022030800': -999.0,
    '2022030801': -999.0,
    '2022030802': -999.0,
    '2022030803': -999.0,
    '2022030804': -999.0,
    '2022030805': -999.0,
    '2022030806': -999.0,
    '2022030807': -999.0,
    '2022030808': -999.0,
    '2022030809': -999.0,
    '2022030810': -999.0,
    '2022030811': -999.0,
    '2022030812': -999.0,
    '2022030813': -999.0,
    '2022030814': -999.0,
    '2022030815': -999.0,
    '2022030816': -999.0,
    '2022030817': -999.0,
    '2022030818':